In [6]:
from typing import List
from IPython.display import display, Math, Latex

def bb(n, bits=7):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    return ('{0:0' + str(bits) + 'b}').format(n & mask)
  
def bn(n, bits=7, underline=None, sep=None):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    arr = list(('{0:0' + str(bits) + 'b}').format(n & mask))
    
    arr.reverse()
    if underline is not None: arr[underline] = r'\underline{' + arr[underline] + '}'
    if sep is not None: arr.insert(sep, '|')
    arr.reverse()
    return '\ '.join(arr)
 

class Mul():
  table: List[List[str]]
  
  def __init__(self, a: int, b: int, correction=True) -> None:
    self.a = a
    self.b = b
    self.table = [
      [
        r'N\ \textup{шага}', 
        r'\textup{Операнды\ и\ действия}',
        r'\begin{array}{c}\textup{СЧП}\ (\textup{старшие}\\\textup{разряды})\end{array}', 
        r'\begin{array}{c}\textup{Множитель}\ и\\\textup{СЧП}\ (\textup{младшие}\\\textup{разряды})\end{array}', 
        r'\textup{Пояснения}'
        ]
    ]
    self.top = ''
    self.bot = ''
    self.process(a, b, correction=correction)
  
  def __str__(self) -> str:
    return self.print()
    
  def print(self):
    la = r'[A]_{пр.}' if self.a > 0 else  r'[-A]_{доп.}'
    lb = r'[B]_{пр.}' if self.b > 0 else  r'[-B]_{доп.}'
    la += '=' + bb(self.a)
    lb += '=' + bb(self.b)
    self.top = (
      '(A' + ('>' if self.a > 0 else '<') + '0,\ B' + ('>' if self.b > 0 else '<') + '0)')
    self.top += r' \\ ' + '\n'
    self.top += '\left.' + la + ';\ ' + lb + '\\right.'
    
    res = r'\begin{array}{c}'
    res += self.top
    res += r' \\ '  + '\n' + r' \\ '
    res += (
      r'\begin{array}{|c|c|c|c|p{5cm}|} \hline ' +
      (r' \\ \hline ' + '\n').join(' & '.join(row) for row in self.table) +
      r' \\ \hline ' + '\n' + r' \end{array} \\' + '\n'
      )
    res += r' \\ ' + '\n' + r' \\ '
    res += self.bot
    res += r'\end{array}'
    
    return f'$${res}$$'
    # display(Math(res))
    
  def process(self, a: int, b: int, width=7, correction=True):
    la = r'[A]_{\textup{пр.}}' if a > 0 else  r'[-A]_{\textup{доп.}}'
    c = int(bb(b), 2)
    step = 0
    last = 0
    shift = 0
    self.table.append([
      str(step),
      r'\textup{СЧП}',
      bn(c >> width),
      bn(c, underline=0),
      r'\textup{Обнуление\ старших\ разрядов\ СЧП}'
      ])
    
    while True:
      step += 1
      shift += 1
      
      if correction:
        if (c >> (shift - 1)) & 1 == 0:
          self.table.append([
            str(step), r'\textup{СЧП} \rightarrow', bn(c >> (width + shift)), bn(c >> shift, underline=0, sep=width-step), r'\textup{Модифицированный\ сдвиг\ СЧП\ и\ множителя\ вправо}'
          ])
        else:
          if a > 0:
            c += (a << (width + step - 1))
          else:
            c += (int(bb(a), 2) << (width + step - 1))
            c |= (int(bb(-1, 16), 2) << ((width * 2) + step - 1))
          self.table.append([
            str(step),
            r'\begin{array}{c} ' + la + r'\\ \textup{СЧП}\\\textup{СЧП}\rightarrow\end{array}',
            r'\begin{array}{c} ' + bn(a) + r' \\ \hline ' + bn(c >> (width + shift - 1)) + r' \\ ' + bn(c >> (width + shift)) +  r' \end{array}',
            r'\begin{array}{c} ' + r' \\ ' + bn(c >> (shift - 1),  sep=width-step+1) + r' \\ ' + bn(c >> (shift), underline=0,  sep=width-step) +  r' \end{array}',
          ( r'\begin{array}{p{3cm}}\textup{Сложение\ СЧП\ с\ множимым}\\\ \textup{Модифицированный\ сдвиг\ СЧП\ и\ множителя\ вправо}\end{array}' if a > 0 else 
            r'\begin{array}{p{3cm}}\textup{Вычитание\ из\ СЧП\ множимого}\\\textup{Модифицированный\ сдвиг\ СЧП\ и\ множителя\ вправо}\end{array}')
          ])
      else:
        cur = int(bb(c >> (shift - 1), width)[-1])
        if cur == last:
          self.table.append([
            str(step), r'\textup{СЧП} \rightarrow', bn(c >> (width + shift)), bn(c >> shift, underline=0, sep=width-step), r'\textup{При\ сдвиге\ младший\ разряд\ не\ изменился;\ Сдвиг\ СЧП\ и\ множителя\ вправо}'
          ])
        else:
          if (cur == 0) ^ (a < 0):
            c += (abs(a) << (width + step - 1))
          else:
            c ^= (int(bb(-1, 8), 2) << ((width * 2) + step))
            c += (int(bb(-abs(a), 8), 2) << (width + step - 1))
          if step == width:
            c = ((a * b) << width) | int(bb(b, 7), 2)
          self.table.append([
            str(step),
            r'\begin{array}{c} ' + (r'[-A]_' if (a < 0) else  r'[A]_') + (r'{\textup{пр.}}' if (cur == 0) ^ (a < 0) else  r'{\textup{доп.}}') + r'\\ \textup{СЧП}\\\textup{СЧП}\rightarrow\end{array}',
            r'\begin{array}{c} ' + (bn(-a) if (cur == 0) ^ (a < 0) else bn(a)) + r' \\ \hline ' + bn(c >> (width + shift - 1)) + r' \\ ' + bn(c >> (width + shift)) +  r' \end{array}',
            r'\begin{array}{c} ' + r' \\ ' + bn(c >> (shift - 1),  sep=width-step+1) + r' \\ ' + bn(c >> (shift), underline=0,  sep=width-step) +  r' \end{array}',
          ( r'\textup{Cложение\ СЧП\ с\ множимым.;\ Сдвиг \ СЧП\ и\ множителя\ вправо}' if (cur == 0) ^ (a < 0) else 
            r'\textup{Вычитание\ множимого\ из\ СЧП.;\ Сдвиг \ СЧП\ и\ множителя\ вправо}')
          ])
        last = cur
      if step == width:
        break
      
    if b < 0 and correction:
      step += 1
      shift += 1
      la = r'[A]_{\textup{пр.}}' if a < 0 else  r'[-A]_{\textup{доп.}}'

      a = -a
      if a > 0:
        c += (a << (width + step - 1))
      else:
        c += (int(bb(a), 2) << (width + step - 1))
        c |= (int(bb(-1, width * 2), 2) << ((width * 2) + step - 1))
      a = -a
      
      # if a > 0:
      # else:
      #   c += (int(bb(a), 2) << (width + step - 1))
      #   c |= (int(bb(-1, 16), 2) << ((width * 2) + step - 1))
        
      if b < 0:
        self.table.append([
          str(step),
          r'\begin{array}{c} ' + la + r'\\ \textup{СЧП}\end{array}',
          r'\begin{array}{c} ' + bn(a) + r' \\ \hline ' + bn(c >> (width + shift - 1))  +  r' \end{array}',
          r'\begin{array}{c} ' + r' \\ ' + bn(c >> (shift - 1),  sep=width-step+1) + r' \end{array}',
          r'\textup{Коррекция\ СЧП}'
        ])

    c >>= width
    c = int(bb(c, width * 2), 2)
    if c > (1 << (width * 2 - 1)): c -= (1 << (width * 2))
    print('calc:', c, 'real:', a * b)
    print(bb(c, 16))
    print(bb(a * b, 16))
    if c != a * b:
      print('PANIC!!!!')
      # raise NameError('SUUUSS')
    
    print()
    self.bot = r' \left[C\right]_{\textup{пр.}}' if a * b > 0 else  r' \left[C\right]_{\textup{доп}.}'
    self.bot += '='
    self.bot += bb( a * b, width * 2) + '_2 = '
    self.bot += str(a * b) + '_{10}'
    
open('res1.tex', 'w+', encoding='utf-8').write(f'''
  {Mul(56, 91)}
  {Mul(-56, 91)}
  {Mul(56, -91)}
  {Mul(-56, -91)}
''')
    
open('res2.tex', 'w+', encoding='utf-8').write(f'''
  {Mul(56, 91, correction=False)}
  {Mul(-56, 91, correction=False)}
  {Mul(56, -91, correction=False)}
  {Mul(-56, -91, correction=False)}
''')

# Mul(41, 40, correction=False)
# Mul(56, -91, correction=False)
# Mul(-56, -91, correction=False)

# display(Math(r'''
#           \begin{array}{|c|c|}
#             a & b
#           \end{array}
#                '''))

calc: 5096 real: 5096
0001001111101000
0001001111101000

calc: -5096 real: -5096
1110110000011000
1110110000011000

calc: -5096 real: -5096
1110110000011000
1110110000011000

calc: 5096 real: 5096
0001001111101000
0001001111101000

calc: 5096 real: 5096
0001001111101000
0001001111101000

calc: -5096 real: -5096
1110110000011000
1110110000011000

calc: -5096 real: -5096
1110110000011000
1110110000011000

calc: 5096 real: 5096
0001001111101000
0001001111101000



12000